# 2. 배깅과 페이스팅
다양한 분류기를 만드는 한 가지 방법은 각기 다른 훈련 알고리즘을 사용하는 것.  
또 다른 방법은 **같은 알고리즘을 사용하되 학습데이터셋의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습**하는 것.
> **배깅** : 이때 학습데이터셋에서 중복을 허용하여 샘플링하는 방식  
**페이스팅** : 학습데이터셋에서 중복을 허용하지 않고 샘플링하는 방식  
  
배깅과 페이스팅 모두 같은 학습 샘플을 여러 예측기에 걸쳐 사용할 수 있음.  
하지만 배깅만 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링할 수 있음.  
> 모든 예측기가 학습을 마치면 앙상블은 모든 예측기의 예측을 모아 새로운 샘플에 대한 예측을 만듦.  
수집 함수는 전형적으로 분류일 때는 통계적 최빈값(직접투표), 회귀일 때는 평균을 계산함  
만약 사용되는 분류기가 클래스 확률을 추정할 수 있으면(predict_proba()가 있으면) 자동으로 간접투표를 사용함

개별 예측기는 원본 학습데이터셋으로 학습시킨 것보다 훨씬 편향되어 있지만 수집함수를 통과하면  
**편향과 분산이 모두 감소함**  
일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 학습할 때와 비교해서 편향은 비슷하지만 **분산은 줄어듦**

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

x, y = make_moons(n_samples=500, noise=0.3, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=500,
max_samples=100, bootstrap=True, n_jobs=-1)

**n_estimators** : 사용할 예측기의 갯수  
**max_samples** : 각 예측기가 학습데이터셋에서 샘플링할 수  
**bootstrap** : 샘플에서 중복을 허용할지 여부 (False일 경우엔 페이스팅이 됨)  
**n_jobs** : 사이킷런이 훈련과 예측에 사용할 CPU 코어수 (-1이면 가능한 모든 코어 사용)

In [7]:
bag_clf.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1)

In [8]:
y_pred = bag_clf.predict(x_test)

---
## oob 평가
배깅을 사용하면 각 예측기는 지정한 숫자의 샘플로만 학습하고 샘플링되지 않은 샘플은 가만히 둠.  
이 샘플링되지 않은 샘플들을 **oob** 샘플이라고 함. 이 oob샘플을 검증세트처럼 이용할 수 있음.

In [10]:
bag_clf = BaggingClassifier(
DecisionTreeClassifier(), n_estimators=500,
max_samples=100, bootstrap=True, n_jobs=-1,
oob_score=True)

In [14]:
bag_clf.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, oob_score=True)

In [15]:
bag_clf.oob_score_

0.9253333333333333

In [17]:
bag_clf.oob_decision_function_

array([[0.36434109, 0.63565891],
       [0.45454545, 0.54545455],
       [1.        , 0.        ],
       [0.0026178 , 0.9973822 ],
       [0.01866667, 0.98133333],
       [0.11842105, 0.88157895],
       [0.4       , 0.6       ],
       [0.06701031, 0.93298969],
       [0.96354167, 0.03645833],
       [0.83914209, 0.16085791],
       [0.54683544, 0.45316456],
       [0.04569892, 0.95430108],
       [0.73045822, 0.26954178],
       [0.85824742, 0.14175258],
       [0.904     , 0.096     ],
       [0.09448819, 0.90551181],
       [0.03957784, 0.96042216],
       [0.91002571, 0.08997429],
       [0.68062827, 0.31937173],
       [0.9343832 , 0.0656168 ],
       [0.05141388, 0.94858612],
       [0.21963824, 0.78036176],
       [0.91578947, 0.08421053],
       [0.97135417, 0.02864583],
       [0.95526316, 0.04473684],
       [0.00263852, 0.99736148],
       [0.93198992, 0.06801008],
       [0.9973822 , 0.0026178 ],
       [0.02493766, 0.97506234],
       [0.6984127 , 0.3015873 ],
       [0.

In [18]:
bag_clf.oob_decision_function_.shape

(375, 2)

---
## 랜덤 패치와 랜덤 서브스페이스
BaggingClassifier는 특성 샘플링도 지원함.  
샘플링은 **max_features, bootstrap_features** 로 조절됨. 작동방식은 max_samples, bootstrap과 동일함.  
> 이를 통해 각 예측기는 무작위로 선택한 입력 특성의 일부분으로만 학습함  

이 기법은 특히 (이미지와 같은) 매우 고차원의 데이터셋을 다룰 때 유용함.  
- **랜덤 패치 방식**: 특성과 샘플을 모두 샘플링하는 방식  
- **랜덤 서브스페이스 방식** : 샘플은 모두 사용하고(max_samples=1.0, bootstrap=False), 특성은 샘플링하는(max_features=1.0보다 작게, bootstrap_features=True) 방식  
  
>**특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춤**